In [5]:
# Install required libraries
%pip install ipykernel
%pip install scipy
%pip install pandas
%pip install nltk
%pip install gensim
%pip install transformers torch

# Import necessary libraries
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel
import torch

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /Users/eunsoe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/eunsoe/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/eunsoe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/eunsoe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/eunsoe/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [7]:
# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

print("Environment setup complete!")

# File paths
filepaths = [
    'organized_blow_molding_basics.csv',
    'Dupont_Table7.csv',
    'organized_pet_bottle_faults.csv',
    'organized_stressesinpebottles.csv',
    'organized_Dupont_blow_moulding_manual.csv'
]

# Potential column names to search for
potential_columns = ["text", "Fault","Cause","Solution", "contents","description","cause","solution"]

# Function to find a matching column in a DataFrame
def find_matching_column(df, potential_columns):
    for column in potential_columns:
        if column in df.columns:
            return column
    return None

# Function to load and concatenate text data from each CSV file
def load_text_from_csv(filepaths, potential_columns):
    text_data = []
    for filepath in filepaths:
        df = pd.read_csv(filepath)
        # Find a matching column
        matching_column = find_matching_column(df, potential_columns)
        if matching_column:
            print(f"Using column '{matching_column}' from {filepath}")
            text_data.extend(df[matching_column].dropna().tolist())
        else:
            print(f"No matching columns found in {filepath}. Skipping this file.")
    return text_data

# Load and concatenate text data from CSV files
texts = load_text_from_csv(filepaths, potential_columns)

# If no text data is found, exit the script
if not texts:
    raise ValueError("No text data found. Please ensure your CSV files contain at least one of the specified columns.")

# Preprocessing function
def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stop words and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in stop_words]
    return tokens

# Preprocess all text data
processed_texts = [preprocess_text(text) for text in texts]

Environment setup complete!
Using column 'text' from organized_blow_molding_basics.csv
Using column 'Fault' from Dupont_Table7.csv
Using column 'contents' from organized_pet_bottle_faults.csv
Using column 'text' from organized_stressesinpebottles.csv
Using column 'text' from organized_Dupont_blow_moulding_manual.csv


In [8]:
# Word2Vec embeddings
word2vec_model = Word2Vec(sentences=processed_texts, vector_size=100, window=5, min_count=1, workers=4)
print("Word2Vec Embedding for 'example':", word2vec_model.wv['example'])  # Replace 'example' with a word in your dataset

Word2Vec Embedding for 'example': [-0.00346718  0.0153346  -0.00734301  0.00390369  0.01308025 -0.03801154
  0.02109214  0.04084329 -0.03065893 -0.00444414 -0.00733983 -0.04682827
 -0.01445073 -0.00266042  0.01384743 -0.01099376 -0.00060612 -0.03428381
 -0.00755478 -0.0418107   0.00405846  0.01831181  0.00233211 -0.01809228
 -0.0182854   0.01144754 -0.02374765 -0.01822493 -0.01053287  0.00116501
  0.03896708  0.00126512  0.00843542 -0.02098665  0.00825326  0.04091962
  0.01460643 -0.01533224 -0.01623146 -0.05094523  0.01019444 -0.033719
 -0.01373419  0.0091665   0.01273148 -0.00044745 -0.02686561  0.00286034
  0.01464302  0.00754794  0.01904545 -0.02654892  0.00821705  0.00149926
 -0.0228466   0.01706906  0.01662044  0.00104531 -0.03452067  0.00330353
 -0.00725799  0.0139695   0.00111371  0.00292858 -0.02780066  0.02553359
  0.01065573  0.0153778  -0.02128258  0.03058524 -0.02894402  0.00472474
  0.02909185  0.00404498  0.01938868  0.0002906  -0.0065371   0.00196924
 -0.01089398  0.011

In [9]:
# BERT embeddings using Hugging Face's transformers
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = bert_model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Sentence-level embedding
    return embeddings

# Generate BERT embeddings for each preprocessed text
bert_embeddings = [get_bert_embeddings(" ".join(tokens)) for tokens in processed_texts]

In [ ]:
# Display an example
print("BERT Embedding for first document:", bert_embeddings[0].detach().numpy())

BERT Embedding for first document: [[-2.56472439e-01  2.66984910e-01  4.50218946e-01  2.75551584e-02
   2.88136274e-01  3.49194109e-02 -1.69095173e-01  1.38960287e-01
  -9.70809441e-03 -3.12174648e-01  6.65363595e-02 -2.69810885e-01
   7.06839636e-02  3.44639093e-01 -1.99757248e-01  2.85503656e-01
   1.12568803e-01 -8.52116868e-02 -2.69523144e-01  3.17722052e-01
   3.71825874e-01 -3.41572553e-01  3.57852638e-01  2.83628494e-01
   2.42721617e-01 -2.76007950e-01 -4.33677882e-02 -4.30727303e-01
   1.09020032e-01  7.63042942e-02  2.56296843e-01  1.09131366e-01
  -3.31082451e-03 -3.76776308e-01  3.23703229e-01 -1.46121949e-01
  -8.31515864e-02  7.15436786e-02 -1.07877821e-01  2.25368053e-01
  -2.01167107e-01 -2.55086035e-01  1.18501008e-01 -1.34477317e-01
  -1.56888768e-01 -7.28874803e-02 -2.05154791e-01 -4.30477709e-02
  -2.23626956e-01 -1.81820035e-01 -2.81188399e-01 -5.10779135e-02
   2.30958745e-01  1.17265038e-01  1.28483444e-01  4.93543655e-01
  -6.75144941e-02 -2.31217712e-01 -2.1631